In [15]:
# imports

import pandas as pd
import requests
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [16]:
# Load csv file from local machine

data = pd.read_csv('C:\\Users\\affuy\\Documents\\Data_Sets\\df_2.csv')
# View make up and structure of the file
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394 entries, 0 to 393
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    394 non-null    int64  
 1   station_name  394 non-null    object 
 2   Latitude      394 non-null    float64
 3   Longitude     394 non-null    float64
 4   Total Bikes   394 non-null    int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 15.5+ KB


In [17]:
# Visualize the DataFrame
data.head(5)

,Unnamed: 0,station_name,Latitude,Longitude,Total Bikes
0,0,2 - Miguel Moya,40.420589,-3.705842,27
1,1,3 - Plaza Conde Suchil,40.430294,-3.706917,19
2,2,6 - Colegio Arquitectos,40.424148,-3.698447,19
3,3,7 - Hortaleza,40.425191,-3.697771,19
4,4,9 - Plaza de San Miguel,40.415606,-3.709508,24


In [18]:
# Create a new column concatenating latitude and longitude into a comma-separated string
data['ll'] = data['Latitude'].astype(str) + ',' + data['Longitude'].astype(str)
data.head()


,Unnamed: 0,station_name,Latitude,Longitude,Total Bikes,ll
0,0,2 - Miguel Moya,40.420589,-3.705842,27,"40.4205886,-3.7058415"
1,1,3 - Plaza Conde Suchil,40.430294,-3.706917,19,"40.4302937,-3.7069171"
2,2,6 - Colegio Arquitectos,40.424148,-3.698447,19,"40.424148,-3.698447"
3,3,7 - Hortaleza,40.425191,-3.697771,19,"40.4251906,-3.6977715"
4,4,9 - Plaza de San Miguel,40.415606,-3.709508,24,"40.4156057,-3.7095084"


In [19]:
# Drop unwanted columns
updated_data = data.drop(['Unnamed: 0', 'Latitude', 'Longitude'], axis=1)

updated_data

,station_name,Total Bikes,ll
0,2 - Miguel Moya,27,"40.4205886,-3.7058415"
1,3 - Plaza Conde Suchil,19,"40.4302937,-3.7069171"
2,6 - Colegio Arquitectos,19,"40.424148,-3.698447"
3,7 - Hortaleza,19,"40.4251906,-3.6977715"
4,9 - Plaza de San Miguel,24,"40.4156057,-3.7095084"
...,...,...,...
389,"598 - Calle San Modesto, 42",24,"40.4860555,-3.692627"
390,"494 - Arturo Soria, 330",23,"40.48038551,-3.66724784"
391,272 - Calle del tejo,23,"40.3980005,-3.6780275"
392,"559 - Calle Babilonia, 19",23,"40.4675299,-3.5882347"


In [20]:
# Make the requst on Foursquare QUERY PARAMS, inputing the ll=40.4205886,-3.7058415, radius 1000, categories 13000

import requests
import os

api_key = os.environ["FOURSQUARE_API_KEY"]

url = "https://api.foursquare.com/v3/places/search?ll=40.4205886%2C-3.7058415&radius=1000&categories=13000&sort=RATING&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": api_key
}

response = requests.get(url, headers=headers)


response = response.json()
response


{'results': [{'fsq_id': '57756e14498e605edcad54fc',
   'categories': [{'id': 13009,
     'name': 'Cocktail Bar',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_',
      'suffix': '.png'}},
    {'id': 13057,
     'name': 'Gastropub',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/gastropub_',
      'suffix': '.png'}},
    {'id': 13347,
     'name': 'Tapas Restaurant',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tapas_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 830,
   'geocodes': {'drop_off': {'latitude': 40.414873, 'longitude': -3.699567},
    'main': {'latitude': 40.414875, 'longitude': -3.699518},
    'roof': {'latitude': 40.414875, 'longitude': -3.699518}},
   'link': '/v3/places/57756e14498e605edcad54fc',
   'location': {'address': 'Calle Echegaray, 21',
    'admin_region': 'Comunidad de Madrid',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Calle Echegaray

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [21]:
import pandas as pd

def categorize_venues(categories):
    # Define category groupings
    category_groups = {
        'Cocktail Bar': 'Bars',
        'Gastropub': 'Bars',
        'Nightclub': 'Nightclubs',
        'Restaurant': 'Restaurants'
    }
    
    for category in categories:
        category_name = category.get('name', 'Other')
        if category_name in category_groups:
            return category_groups[category_name]
    
    return 'Other'  # If no matching category found, assign to 'Other' category

def extract_data(response):
    data_list = []
    nearby_points_of_interest = []

    for result in response['results']:
        name = result.get('name', 'N/A')
        
        # Extracting location information
        location_info = result.get('location', {})
        address = location_info.get('address', 'N/A')
        city = location_info.get('locality', 'N/A')
        country = location_info.get('country', 'N/A')
        
        # Extracting ranking information (if available)
        ranking_info = result.get('ranking', {})
        ranking = ranking_info.get('rank', 'N/A')

        # Extracting categories
        categories = result.get('categories', [])
        
        # Categorize the venue based on categories
        category = categorize_venues(categories)

        data_list.append({'Name': name, 'Address': address, 'City': city, 'Country': country, 'Ranking': ranking, 'Categories': category})
        
        # Extracting nearby points of interest
        nearby_info = result.get('nearby_points_of_interest', [])
        nearby_points_of_interest.extend(nearby_info)

    # Create a DataFrame from the extracted data
    df_fs = pd.DataFrame(data_list)
    
    # Create a DataFrame for nearby points of interest
    poi_df = pd.DataFrame(nearby_points_of_interest, columns=['Point of Interest'])
    
    return df_fs, poi_df

# Assuming you have your response stored in a variable called 'response'
df_fs, poi_df = extract_data(response)

# Display the resulting DataFrames
print("Venues:")
print(df_fs)

print("\nNearby Points of Interest:")
print(poi_df)


Venues:
                              Name                       Address    City  \
0                      Salmon Guru           Calle Echegaray, 21  Madrid   
1                       La Primera             Calle Gran Vía, 1  Madrid   
2                       HanSo Café             Calle del Pez, 20  Madrid   
3                    La Pescadería            Calle Ballesta, 32  Madrid   
4                          Amorino               Calle Mayor, 40  Madrid   
5                         La Mayor               Calle Mayor, 77  Madrid   
6              Bodega de la Ardosa               Calle Colón, 13  Madrid   
7                     Rosi la Loca             Calle de Cádiz, 4  Madrid   
8   Azotea Círculo de Bellas Artes               Calle Alcalá 42  Madrid   
9                        Valdemeso           Mesonero Romanos, 8  Madrid   
10        Arrocería Marina Ventura  Calle Ventura de la Vega, 13  Madrid   
11                360° Rooftop Bar            Calle Gran Vía, 84  Madrid   
12  

Put your parsed results into a DataFrame

In [22]:
# Create a DataFrame from the extracted data
df_fs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        20 non-null     object
 1   Address     20 non-null     object
 2   City        20 non-null     object
 3   Country     20 non-null     object
 4   Ranking     20 non-null     object
 5   Categories  20 non-null     object
dtypes: object(6)
memory usage: 1.1+ KB


In [23]:
# Save this as a csv file

import pandas as pd

# Save the DataFrame as a CSV file
df_fs.to_csv('C:\\Users\\affuy\\Documents\\Data_Sets\\df_foursquare.csv', index=False)


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [24]:
import requests
import os

yelp_api_key = os.environ["YELP_API_KEY"]
url = "https://api.yelp.com/v3/businesses/search?latitude=40.4205886&longitude=-3.7058415&radius=1000&categories=restaurants%2Cbars%2Clandmarks&sort_by=rating&limit=20"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {yelp_api_key}"  # Replace with your actual Yelp API access token
}

response = requests.get(url, headers=headers)

response = response.json()

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [25]:
def parse_yelp_response(response):
    businesses = response.get('businesses', [])
    parsed_data = []

    for business in businesses:
        name = business.get('name', '')
        location = ', '.join(business.get('location', {}).get('display_address', []))
        ranking = business.get('rating', 0)
        categories = ', '.join([category.get('title', '') for category in business.get('categories', [])])

        parsed_data.append({
            'Name': name,
            'Location': location,
            'Ranking': ranking,
            'Category': categories
        })

    df = pd.DataFrame(parsed_data)
    return df

Put your parsed results into a DataFrame

In [26]:
# Data already passed into a DataFrame in the function above
# We call the function to view the DataFrame
df = parse_yelp_response(response)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

##### I have found that both the Foursquare and Yelp APIs provide valuable data for various purposes. The choice of which one to use depends on your specific needs and the type of data you require. 

##### The Yelp API has been particularly helpful for me. It provided me with information primarily related to businesses and places, with a strong focus on user-generated reviews and ratings. This includes detailed information about businesses such as their names, addresses, contact details, ratings, reviews, and categories. It has proven to be very useful when I needed to analyze and compare user opinions, ratings, and reviews about restaurants, bars, and other businesses. I've found it to be more suitable for applications related to the hospitality and food service industry.

##### On the other hand, the Foursquare API has also been quite valuable in my work. It offers a broader range of location-based data and insights beyond just businesses and places. In addition to business details, Foursquare has provided me with data related to check-ins, user-generated tips, user profiles, and location trends. This extensive dataset has been instrumental in exploring user behavior, tracking trends in check-in activity, and understanding user preferences in a more general sense. It's a versatile choice for applications that require a deeper understanding of user engagement and location-based behavior.

##### In summary, if you primarily need detailed information about businesses, their ratings, and reviews, the Yelp API may be more suitable for your needs. On the other hand, if you're interested in a broader range of location-based data, including user activity and trends, the Foursquare API can provide valuable insights. Your choice should align with your specific analytical goals and the type of data that best serves your analysis."

Get the top 10 restaurants according to their rating

In [27]:
# Sort the DataFrame by 'Ranking' in ascending order
sorted_df = df.sort_values(by='Ranking', ascending=False)

# Display the top 10 results
top_10_df = sorted_df.head(10)

# Rename the index column to 'top_ranking'
top_10_df.index.name = 'top_ranking'

# Print the top 10 DataFrame
print(top_10_df)


                                              Name  \
top_ranking                                          
0                                    Sabe a Gloria   
1                                     Blow Dry Bar   
18                                    Pez Tortilla   
17           Arquibar - La Traviesa de Conde Duque   
16                                            Bico   
15                                         Roostiq   
14                                Vinoteca Moratín   
13                                          Araldo   
12                                     Baton Rouge   
11                                       Kuoco 360   

                                                    Location  Ranking  \
top_ranking                                                             
0              Calle de las Huertas, 53, 28014 Madrid, Spain      5.0   
1                   Calle de Pelayo, 76, 28004 Madrid, Spain      5.0   
18                  Calle Cava Baja, 42, 28005 Madrid, Spai

In [28]:
# Save this as a csv file

import pandas as pd

# Save the DataFrame as a CSV file
df.to_csv('C:\\Users\\affuy\\Documents\\Data_Sets\\df_Yelp.csv', index=False)
